In [1]:
#pip install implicit

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from implicit.als import AlternatingLeastSquares

c:\Users\Matias\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\Matias\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [ ]:
# Load dataset
file_path = "C:/Users/Matias/Desktop/TrabajoFinal_MasterBigData/00_Data_Bases/Cluster5_1_items.csv"  # Update with correct path if needed
df = pd.read_csv(file_path)

In [4]:
# Prepare the user-item interaction matrix
user_item_matrix = df.pivot_table(index='user_id', columns='product_id', values='reordered', fill_value=0)

# Convert to sparse matrix format
sparse_user_item = csr_matrix(user_item_matrix.values)

In [5]:
# Split data into training and validation sets
train_data, test_data = train_test_split(user_item_matrix, test_size=0.2, random_state=42)
train_sparse = csr_matrix(train_data.values)
test_sparse = csr_matrix(test_data.values)

In [6]:
# Train ALS model
als_model = AlternatingLeastSquares(factors=50, regularization=0.1, iterations=20)
als_model.fit(train_sparse)


c:\Users\Matias\anaconda3\lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: OpenBLAS is configured to use 16 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/20 [00:00<?, ?it/s]

In [7]:
# Function to get recommendations
def get_recommendations(user_id, model, user_item_matrix, sparse_data, n=5):
    if user_id not in user_item_matrix.index:
        raise ValueError(f"User ID {user_id} not found in dataset.")
    
    user_idx = user_item_matrix.index.get_loc(user_id)  # Get user index
    recommendations = model.recommend(user_idx, sparse_data[user_idx], N=n)
    
    # Convert ALS product indices back to original product IDs
    product_indices = [i for i, _ in recommendations]
    product_ids = user_item_matrix.columns[product_indices].tolist()
    scores = [score for _, score in recommendations]
    
    return list(zip(product_ids, scores))

In [ ]:
# Example: Get top 5 recommendations for a specific user
user_id_example = df['user_id'].iloc[0]

In [ ]:
# Ensure user ID exists in matrix
if user_id_example not in user_item_matrix.index:
    raise ValueError(f"User ID {user_id_example} not found in dataset.")

print("User ID Example:", user_id_example)
print("User Item Matrix Shape:", user_item_matrix.shape)  # Replace with actual user ID
recommendations = get_recommendations(user_id_example, als_model, user_item_matrix, sparse_user_item[user_item_matrix.index.get_loc(user_id_example)])

# Display results
print("Recommendations for user:", recommendations)


User ID Example: 58222
User Item Matrix Shape: (37973, 134)


IndexError: row index (10810) out of range